In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install madmom
!pip install ffmpeg

In [ ]:
data_folder =  "/content/drive/MyDrive/11785-IDL/11785 Project/data"

In [ ]:
from scipy import io
import os
import numpy as np
import mne
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
# from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torchaudio
# import wandb 
import madmom
import madmom.features.beats
import madmom.features.onsets
import mir_eval
import ffmpeg
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc
import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime



ModuleNotFoundError: ignored

##Load the files

In [ ]:
new_data = np.empty((10,20,125,37500))


In [ ]:
# data = loaded['data21']
for k in range(21,31):
    print(k)
    loaded = io.loadmat(data_folder +'/songs_mat/song'+str(k)+'_Imputed.mat',  struct_as_record=True)
    print(loaded['data'+str(k)].shape)
    data = np.transpose(np.array(loaded['data'+str(k)]), (2,0,1))
    nsubjects, nchannels, ntimesteps = data.shape
    pad_length = 37500-ntimesteps
    EEGdata = np.pad(data, ((0,0) , (0,0), (0, pad_length)), constant_values=0)
    new_data[k-21,:,:,:] = EEGdata
# EEGdata.shape

21
(125, 34795, 20)
22
(125, 33891, 20)
23
(125, 34469, 20)
24
(125, 36797, 20)
25
(125, 36114, 20)
26
(125, 34478, 20)
27
(125, 36454, 20)
28
(125, 36513, 20)
29
(125, 36712, 20)
30
(125, 37221, 20)


##Writing out the files

In [ ]:
for u in range(20):
    mdict={'data':new_data[:,u,:,:],
          'user': 'User_'+str(u)}
    io.savemat(data_folder+'/users_mat/user'+str(u)+'_restructured.mat',mdict)

#Restructuring the Audio Files


In [ ]:
folder_path = data_folder + '/songs_wav/'
song_files = os.listdir(folder_path)# TODO: list files in the mfcc directory
song_files = [s for s in song_files if 'wav' in s]
song_files.sort()
timestamp=[]
onehot = np.zeros((10,37500))

In [ ]:
song_files

['01) first_fires.wav',
 '02) oino.wav',
 '03) tiptoes.wav',
 '04) careless_love.wav',
 '05) lebanese_blonde.wav',
 '06) canopee.wav',
 '07) doing_yoga.wav',
 '08) until_the_sun_needs_to_rise.wav',
 '09) silent_shout.wav',
 '10) the_last_thing_you_should_do.wav']

In [ ]:
for f,files in enumerate(song_files):
    print(files)
    proc1 = madmom.features.onsets.OnsetPeakPickingProcessor(fps = 100) #What should fps be?
    act1 = madmom.features.onsets.RNNOnsetProcessor()(folder_path+files) 
    onset_values = proc1(act1) 
    print(onset_values)
    timestamp.append(onset_values)
    # gt = np.zeros(37500)
    idx_list=[]
    for onset in (onset_values):
        idx_list.append(onset*125)
    print(idx_list)
    idx_list = np.array(idx_list)

    idx_list = np.round(idx_list,0).astype('int64')
    print(idx_list)

    onehot[f][idx_list] = 1


##Saving the audio files

In [ ]:
np.save(data_folder+'/ground_truths/one_hot.npy',onehot)
np.save(data_folder+'/ground_truths/timestamps.npy',timestamp,allow_pickle=True)


NameError: ignored